In [72]:
from datasets import load_from_disk, load_dataset
from transformers import LayoutLMv2FeatureExtractor, LayoutXLMTokenizerFast, LayoutXLMProcessor
import numpy as np

In [50]:
%load_ext line_profiler

In [2]:
rvl_cdip = load_dataset('rvl_cdip')

Found cached dataset rvl_cdip (/home/hj36wegi/scratch/data/huggingface/datasets/rvl_cdip/default/1.0.0/ea410993ed3f5b9744d8616ffbaad5f70a75a21a4233626dd07b3de31d381e53)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 70.43it/s]


In [125]:
img_processor = LayoutLMv2FeatureExtractor(do_resize=False,
                                         do_rescale=False,
                                         do_normalize=False,
                                         ocr_lang='eng',
                                         )

/home/hj36wegi/.conda/envs/adapters/lib/python3.11/site-packages/transformers/models/layoutlmv2/feature_extraction_layoutlmv2.py:30: FutureWarning: The class LayoutLMv2FeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use LayoutLMv2ImageProcessor instead.
  warnings.warn(


In [108]:
tokenizer = LayoutXLMTokenizerFast.from_pretrained("SCUT-DLVCLab/lilt-infoxlm-base",return_tensors='pt',return_special_tokens_mask=True)

In [109]:
processor = LayoutXLMProcessor(img_processor,tokenizer)

In [124]:
img = np.array(rvl_cdip['train'][0]['image'].convert('RGB'))
img.dtype

dtype('uint8')

In [126]:
processed = img_processor(rvl_cdip['train'][0]['image'].convert('RGB'))
img2 = processed['pixel_values'][0]

In [129]:
img2.dtype

dtype('uint8')

In [127]:
np.max(img-img2.transpose(1,2,0))

0

In [122]:
def get_ocr(example,index):
    img = example['image'].convert('RGB')
    ocr = img_processor(img)
    ocr.pop('pixel_values')
    ocr['id'] = index
    return ocr

In [54]:
%lprun -f get_ocr get_ocr(rvl_cdip['train'][0])

Timer unit: 1e-09 s

Total time: 0.930947 s
File: /tmp/ipykernel_4084540/1544902331.py
Function: get_ocr at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def get_ocr(example):
     2         1    1491205.0 1491205.0      0.2      img = example['image'].convert('RGB')
     3         1  929442430.0 929442430.0     99.8      ocr = img_processor(img)
     4         1      12694.0  12694.0      0.0      ocr.pop('pixel_values')
     5         1        214.0    214.0      0.0      return ocr

In [123]:
rvl_cdip.map(get_ocr,with_indices=True)

KeyboardInterrupt: 

In [114]:
def process_example(example):
    img = example['image'].convert('RGB')
    encoding = processor(images=[img])
    return encoding

In [83]:
%lprun -f process_example process_example(rvl_cdip['train'][0])

Timer unit: 1e-09 s

Total time: 0.943715 s
File: /tmp/ipykernel_4084540/3647424934.py
Function: process_example at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def process_example(example):
     2         1    1499439.0 1499439.0      0.2      img = example['image'].convert('RGB')
     3         1  942215599.0 942215599.0     99.8      encoding = processor(images=[img])
     4         1        412.0    412.0      0.0      return encoding

In [93]:
enc = process_example(rvl_cdip['train'][0])

In [115]:
enc = rvl_cdip['train'].select(range(3)).map(process_example)

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.20s/ examples]/home/hj36wegi/.conda/envs/adapters/lib/python3.11/site-packages/datasets/features/image.py:319: UserWarning: Downcasting array dtype int64 to uint8 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")


In [116]:
enc.set_format('pt')

In [117]:
enc

Dataset({
    features: ['image', 'label', 'input_ids', 'attention_mask', 'bbox'],
    num_rows: 3
})

In [118]:
for k,v in enc[0].items():
    print(k, v.shape)

image torch.Size([1, 3, 1000, 762])
label torch.Size([])
input_ids torch.Size([1, 196])
attention_mask torch.Size([1, 196])
bbox torch.Size([1, 196, 4])
